Upload data to sqlite tables.

In [1]:
import pandas as pd
import numpy as np
from IPython import display as dis
import scipy.io.wavfile as wav
import tensorflow as tf
from tensorflow.contrib.legacy_seq2seq.python.ops.seq2seq import basic_rnn_seq2seq
from tensorflow.contrib.rnn import RNNCell, LSTMCell, MultiRNNCell
from scipy import signal 
from librosa import core

%matplotlib inline

In [2]:
dis.Audio("dataset/wav/Ses01F_impro01/Ses01F_impro01_F000.wav")

In [3]:
(sig,rate) = core.load("dataset/wav/Ses01F_impro01/Ses01F_impro01_F000.wav", sr = 4000)
print(sig, rate)
print(len(sig))

[-0.00294593 -0.0031436  -0.00048992 ..., -0.0009217  -0.00217147  0.        ] 4000
7783


In [4]:
dis.Audio(data = sig, rate = rate)

In [25]:
class network(object):
    
    time_step = 7783
    hidden_layers = 1
    batch_size = 1
    latent_dim = 61
    
    def __init__(self):
        pass
            
    def build_encoder_layer(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        time_step = self.time_step
        batch_size = self.batch_size
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        
        
        with tf.variable_scope("Input"):
            self.x_input = tf.placeholder("float", shape=[batch_size, time_step, 1])
            self.y_input_ = tf.placeholder("float", shape=[batch_size, time_step, 1])
            
            self.keep_prob = tf.placeholder("float")
            self.lr = tf.placeholder("float")
            
            self.y_list = []
            loss = 0
            
            hidden_state = tf.zeros([batch_size, 1])
            current_state = tf.zeros([batch_size, 1])
            state = hidden_state, current_state
            
            self.x_list = tf.unstack(self.x_input, axis= 1)
            self.y_list_ = tf.unstack(self.y_input_, axis = 1)
            
        with tf.variable_scope("lstm_encoder") as enc_scope:
            lstm = tf.contrib.rnn.BasicLSTMCell(latent_dim)
            #multi_cell = MultiRNNCell([lstm for i in range(hidden_layers)] )
            
            # Encoder
            for _, sig in enumerate(self.x_list):
                output, state = lstm(sig, state)
                enc_scope.reuse_variables()
                #loss += tf.losses.mean_squared_error(output, sig)
                #self.y_list.append(output)
                
        
        with tf.variable_scope("lstm_decoder") as dec_scope:
            lstm = tf.contrib.rnn.BasicLSTMCell(latent_dim)
            #multi_cell = MultiRNNCell([lstm for i in range(hidden_layers)] )
            
            # Decoder
            for y_sig in enumerate(self.y_list_):
                output, state = lstm(output, state)
                dec_scope.reuse_variables()
                loss += tf.losses.mean_squared_error(output, y_sig)
             
            self.y_list.append(output)
            
        with tf.variable_scope("Loss"):
            
            self.pred = tf.stack(self.y_list)
            self.regularized_loss = loss #tf.losses.mean_squared_error(self.pred, self.y_input_)
            
        with tf.variable_scope("Optimizer"):
            learning_rate=self.lr
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        
        # add Saver ops
        self.saver = tf.train.Saver()
        
    def build_decoder_layer(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        time_step = self.time_step
        batch_size = self.batch_size
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        
        
        with tf.variable_scope("Input"):
            self.x_input = tf.placeholder("float", shape=[batch_size, time_step, 1])
            self.y_input_ = tf.placeholder("float", shape=[batch_size, time_step, 1])
            
            self.keep_prob = tf.placeholder("float")
            self.lr = tf.placeholder("float")
            
            self.y_list = []
            loss = 0
            
            hidden_state = tf.zeros([batch_size, 1])
            current_state = tf.zeros([batch_size, 1])
            state = hidden_state, current_state
            
            self.x_list = tf.unstack(self.x_input, axis= 1)
            self.y_list_ = tf.unstack(self.y_input_, axis = 1)
            
        with tf.variable_scope("lstm_encoder") as enc_scope:
            lstm = tf.contrib.rnn.BasicLSTMCell(latent_dim)
            #multi_cell = MultiRNNCell([lstm for i in range(hidden_layers)] )
            
            # Encoder
            for _, sig in enumerate(self.x_list):
                output, state = lstm(sig, state)
                enc_scope.reuse_variables()
                #loss += tf.losses.mean_squared_error(output, sig)
                #self.y_list.append(output)
                
        
        with tf.variable_scope("lstm_decoder") as dec_scope:
            lstm = tf.contrib.rnn.BasicLSTMCell(latent_dim)
            #multi_cell = MultiRNNCell([lstm for i in range(hidden_layers)] )
            
            # Decoder
            for y_sig in enumerate(self.y_list_):
                output, state = lstm(output, state)
                dec_scope.reuse_variables()
                loss += tf.losses.mean_squared_error(output, y_sig)
             
            self.y_list.append(output)
            
        with tf.variable_scope("Loss"):
            
            self.pred = tf.stack(self.y_list)
            self.regularized_loss = loss #tf.losses.mean_squared_error(self.pred, self.y_input_)
            
        with tf.variable_scope("Optimizer"):
            learning_rate=self.lr
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        
        # add Saver ops
        self.saver = tf.train.Saver()
        

In [26]:
import collections

y_pred = None
class Train:    
    
    def train(epochs, net, lrs):
        global y_pred
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            start_time = time.perf_counter()
            
            
            for c, lr in enumerate(lrs):
                for epoch in range(1, (epochs+1)):
                    print("Step {} ".format(train_loss))
                    _, train_loss = sess.run([net.train_op, net.regularized_loss], #net.summary_op
                                              feed_dict={net.x_input: x_train[np.newaxis,...], 
                                                         net.y_input_: y_train[np.newaxis,...],
                                                         net.lr:lr})

                    print("Training Loss: {:.6f}".format(train_loss))

                    #valid_accuracy,valid_loss = sess.run([net.tf_accuracy, net.regularized_loss], #net.summary_op 
                    #                                      feed_dict={net.x_input: x_valid[np.newaxis,...], 
                    #                                                 net.y_input_: y_valid[np.newaxis,...],
                    #                                                 net.lr:lr})
                        
                    accuracy, y_pred = sess.run([net.regularized_loss, 
                                               net.pred], 
                                                feed_dict={net.x_input: x_train[np.newaxis,...], 
                                                         net.y_input_: y_train[np.newaxis,...],
                                                         net.lr:lr})
                                              


In [27]:
import itertools

class Hyperparameters:
    def start_training():
        
        epochs = 1
        lrs = [1e-5]
        n = network()
        n.build_layers()
        Train.train(epochs, n, lrs)
            

In [28]:
sig = np.reshape(sig, (1,-1, 1))
print(sig.shape)

(1, 7783, 1)


In [29]:
x_train = y_train = sig
Hyperparameters.start_training()

ValueError: Trying to share variable lstm_encoder/basic_lstm_cell/kernel, but specified shape (62, 244) and found shape (2, 244).